In [1]:
import pandas as pd
import re
import urllib3 as url
import urllib
import numpy as np
import os
import requests as req
import csv
from requests.utils import requote_uri
from pathlib import Path
http = url.PoolManager()

In [2]:
def downloadCountries(listOfCountries):
    for country in listOfCountries:
        r = http.request('GET', requote_uri('berkeleyearth.lbl.gov/auto/Regional/TAVG/Text/' + country[0] + '-TAVG-Trend.txt'));
        if  Path('data/' + country[0]+'.txt').is_file():
            os.remove('data/' + country[0]+'.txt');
        file = open('data/' + country[0]+'.txt', 'wb');
        file.write(r.data);
        file.close();

In [3]:
r = http.request('GET', 'http://berkeleyearth.lbl.gov/country-list')
#countrytable = re.search('<table class="table table-condensed table-hover">(.)</table>', str(r.data))
countryList = np.array(re.findall('<tr><td><a href="http://berkeleyearth.lbl.gov/regions/(.*?)">(.*?)</td>', r.data.decode()))
countryList[:,0]; # liste der Links
countryList[:,1]; # liste der Namen der Länder #TODO: Encoding von Sonderzeichen in Ländernamen reparieren
#downloadCountries(countrylist);

In [24]:
def loadCountry(file):
    data = pd.read_csv(file, sep="\s+",engine='python', comment = "%", names=["Year","Month","Monthly Anomaly","Monthly Uncertainty","Annual Anomaly","Annual Uncertainty","Five-year Anomaly","Five-year Uncertainty","Ten-year Anomaly","Ten-year Uncertainty","Twenty-year Anomaly","Twenty-year Uncertainty"]);
    data["YearT"]=pd.to_datetime(data[["Year"]].assign(day=1,month=1));
    temperature=open(file, 'r');
    absolute_temp=re.findall(r'Estimated Jan 1951-Dec 1980 absolute temperature \(C\): (-?\d+\.\d+) \+\/- (\d+\.\d+)',str(temperature.read()));
    temperature.close();
    data["Total Temperature"]=data["Annual Anomaly"]+float(absolute_temp[0][0]);
    dataJune = data[data["Month"]==6];
    return dataJune["Annual Anomaly"][-101:]

In [25]:
def getYears():
    data = pd.read_csv('data/afghanistan.txt', sep="\s+",engine='python', comment = "%", names=["Year","Month","Monthly Anomaly","Monthly Uncertainty","Annual Anomaly","Annual Uncertainty","Five-year Anomaly","Five-year Uncertainty","Ten-year Anomaly","Ten-year Uncertainty","Twenty-year Anomaly","Twenty-year Uncertainty"]);
    temperature=open('data/afghanistan.txt', 'r');
    absolute_temp=re.findall(r'Estimated Jan 1951-Dec 1980 absolute temperature \(C\): (-?\d+\.\d+) \+\/- (\d+\.\d+)',str(temperature.read()));
    temperature.close();
    dataJune = data[data["Month"]==6];
    return dataJune["Year"][-101:]

In [26]:
def generateOverview():
    countryData = [];
    #countryData.append(getYears());
    for country in os.listdir('data/'):
        if not (country == '.ipynb_checkpoints'):
            countryData.append(loadCountry('data/' + country));
    df = pd.DataFrame(data=np.array(countryData),columns=getYears());
    files = os.listdir('data/');
    #files.remove('.ipynb_checkpoints');
    files = [os.path.splitext(filename)[0] for filename in files]
    df.index = files;
    df.to_csv('overview.csv',sep=',');

In [27]:
generateOverview()

afghanistan.txt:			101
albania.txt:			101
algeria.txt:			101
american-samoa.txt:			101
andorra.txt:			101
angola.txt:			101
anguilla.txt:			101
antigua-and-barbuda.txt:			101
argentina.txt:			101
armenia.txt:			101
aruba.txt:			101
australia.txt:			101
austria.txt:			101
azerbaijan.txt:			101
bahamas.txt:			101
bahrain.txt:			101
baker-island.txt:			101
bangladesh.txt:			101
barbados.txt:			101
belarus.txt:			101
belgium.txt:			101
belize.txt:			101
benin.txt:			101
bhutan.txt:			101
bolivia.txt:			101
bonaire,-saint-eustatius-and-saba.txt:			101
bosnia-and-herzegovina.txt:			101
botswana.txt:			101
brazil.txt:			101
british-virgin-islands.txt:			101
bulgaria.txt:			101
burkina-faso.txt:			101
burma.txt:			101
burundi.txt:			101
cambodia.txt:			101
cameroon.txt:			101
canada.txt:			101
cape-verde.txt:			101
cayman-islands.txt:			101
central-african-republic.txt:			101
chad.txt:			101
chile.txt:			101
china.txt:			101
christmas-island.txt:			101
colombia.txt:			101
comoros.txt:			101
co